# k-近邻算法一般流程
1. 收集数据，并格式化
2. 距离计算
3. 分析数据
4. 测试算法，即准确度

In [1]:
#科学计算工具包
from numpy import *
#运算符模块
import operator 

def createDataSet():
    #用于创建数据集合标签
    group = array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
    labels = ['A','A','B','B']
    return group,labels
createDataSet()

(array([[1. , 1.1],
        [1. , 1. ],
        [0. , 0. ],
        [0. , 0.1]]), ['A', 'A', 'B', 'B'])

对未知类别属性的数据集中的每个点依次执行以下操作：
1. 计算已知类别数据集中的点与测试点之间的距离。
2. 按照距离递增次序排序。
3. 选取与测试点距离最小的k个点。
4. 确定前k个点所在类别的出现频率。
5. 返回前k个点出现频率最高的类别作为当前点的预测分类。



In [2]:
def classify0(inX,dataSet,labels,k):
    #第n维，即表示列表符号"[]"第几层，从1层开始，第1维表示[a,b,c]的a,b,c等3个项
    #统计第1维项数量，即数据的行数
    dataSetSize = dataSet.shape[0]
    #将输入坐标通过tile，扩大为样本数据的行数，然后每行减去对应行的样本
    diffMat = tile(inX,(dataSetSize,1)) - dataSet
    #求矩阵各项平方
    sqDiffMat = diffMat**2
    #对第2维项求和,即对所有列求和
    sqDistances = sqDiffMat.sum(axis=1)
    #取根号
    distances = sqDistances**0.5
    #按照距离递增排序,返回索引号
    sortedDistIndicies = distances.argsort()
    classCount = {}
    #选择距离最小的k个点
    for i in range(k):
        #排序的前k个点对应的类
        voteIlabel = labels[sortedDistIndicies[i]]
        #字典对每个类的统计
        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1
    #依据字典的值itemgetter(1)，即统计数进行排序
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    #返回最大统计数
    return sortedClassCount[0][0]
group,labels = createDataSet()   
classify0([0,0],group,labels,3)

'B'

# 读取约会匹配特征文本到列表

飞行公里  玩游戏所耗时间百分比   每周冰淇淋体积   喜欢度


In [3]:
def file2matrix(filename):
    fr = open(filename)
    love_dictionary = {'largeDoses':3, 'smallDoses':2, 'didntLike':1}
    arrayOLines = fr.readlines()
    numberOfLines = len(arrayOLines)
    returnMat = zeros((numberOfLines,3))
    classLabelVector = []
    index = 0
    for line in arrayOLines:
        line = line.strip() #清除行前后的空格以及换行符
        listFromLine = line.split('\t') #根据\t分组
        returnMat[index,:] = listFromLine[0:3]# 将前三行赋值
        if listFromLine[-1].isdigit():#矩阵最后一列是否数字
            classLabelVector.append((int(listFromLine[-1])))
        else:
            classLabelVector.append(love_dictionary.get(listFromLine[-1]))
        index += 1
    fr.close()
    return returnMat,classLabelVector
datingDataMat,datingLabels = file2matrix('datingTestSet.txt')
print(datingDataMat)
print(datingLabels)

[[4.0920000e+04 8.3269760e+00 9.5395200e-01]
 [1.4488000e+04 7.1534690e+00 1.6739040e+00]
 [2.6052000e+04 1.4418710e+00 8.0512400e-01]
 ...
 [2.6575000e+04 1.0650102e+01 8.6662700e-01]
 [4.8111000e+04 9.1345280e+00 7.2804500e-01]
 [4.3757000e+04 7.8826010e+00 1.3324460e+00]]
[3, 2, 1, 1, 1, 1, 3, 3, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 2, 3, 2, 1, 2, 3, 2, 3, 2, 3, 2, 1, 3, 1, 3, 1, 2, 1, 1, 2, 3, 3, 1, 2, 3, 3, 3, 1, 1, 1, 1, 2, 2, 1, 3, 2, 2, 2, 2, 3, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 3, 1, 2, 3, 2, 2, 1, 3, 1, 1, 3, 3, 1, 2, 3, 1, 3, 1, 2, 2, 1, 1, 3, 3, 1, 2, 1, 3, 3, 2, 1, 1, 3, 1, 2, 3, 3, 2, 3, 3, 1, 2, 3, 2, 1, 3, 1, 2, 1, 1, 2, 3, 2, 3, 2, 3, 2, 1, 3, 3, 3, 1, 3, 2, 2, 3, 1, 3, 3, 3, 1, 3, 1, 1, 3, 3, 2, 3, 3, 1, 2, 3, 2, 2, 3, 3, 3, 1, 2, 2, 1, 1, 3, 2, 3, 3, 1, 2, 1, 3, 1, 2, 3, 2, 3, 1, 1, 1, 3, 2, 3, 1, 3, 2, 1, 3, 2, 2, 3, 2, 3, 2, 1, 1, 3, 1, 3, 2, 2, 2, 3, 2, 2, 1, 2, 2, 3, 1, 3, 3, 2, 1, 1, 1, 2, 1, 3, 3, 3, 3, 2, 1, 1, 1, 2, 3, 2, 1, 3, 1, 3, 2, 2, 3, 1, 3, 1, 1, 2, 1, 2, 2, 1, 

# Matplotlib散点图

In [4]:
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure() #指定画布
ax = fig.add_subplot(111) #在指定画布中分割为1行1列，选择第1块
ax.scatter(datingDataMat[:,1],datingDataMat[:,2],15.0*array(datingLabels),15.0*array(datingLabels))
plt.show()

<Figure size 640x480 with 1 Axes>

# 归一化数据 
由于飞行公里数相对于其他特征来说，数值非常大，在使用欧式范数公式计算中，飞行公里数严重影响了其他数据表现

故将采取数据归一化，使三个特征值归纳到0到1或-1到1之间。

公式：newvalue = (oldValue -min)/(max - min)

In [5]:
def autoNorm(dataSet):
  minVals = dataSet.min(0) #axis=0;求每列的最小值
  maxVals = dataSet.max(0) #axis=0,求每列的最大值
  ranges = maxVals - minVals 
  normDataSet = zeros(shape(dataSet)) #以dataSet行列设置0值矩阵
  m = dataSet.shape[0] #axis=0，每列的长度
  normDataSet = dataSet - tile(minVals,(m,1)) #矩阵相减在同一size
  normDataSet = normDataSet/tile(ranges,(m,1)) #矩阵相除在同一size
  return normDataSet,ranges,minVals

normMat,ranges,minVals = autoNorm(datingDataMat)
print(normMat)
print(ranges)
print(minVals)

[[0.44832535 0.39805139 0.56233353]
 [0.15873259 0.34195467 0.98724416]
 [0.28542943 0.06892523 0.47449629]
 ...
 [0.29115949 0.50910294 0.51079493]
 [0.52711097 0.43665451 0.4290048 ]
 [0.47940793 0.3768091  0.78571804]]
[9.1273000e+04 2.0919349e+01 1.6943610e+00]
[0.       0.       0.001156]


# 分类器针对约会匹配数据的测试代码
1. 提供已有数据的90%作为训练样本训练分类器
2. 使用10%的随机数据测试分类器
3. 错误率=错误结果/测试总数据

In [6]:
def datingClassTest():
    hoRatio = 0.10
    datingDataMat,datingLabels = file2matrix('datingTestSet.txt')
    normMat,ranges,minVals = autoNorm(datingDataMat)
    m = normMat.shape[0]  #总样本量
    numTestVecs = int(m*hoRatio) #测试数据集抽取hoRatio的量
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:],normMat[numTestVecs:m,:],datingLabels[numTestVecs:m],4) #m-numTestVecs个样本
        print("the classifier came back with:%d, the real answer is: %d" % (classifierResult ,datingLabels[i]))
        if (classifierResult != datingLabels[i]):
            errorCount += 1.0
            print("the total error rate is:%f" % (errorCount/float(numTestVecs)))
datingClassTest()
    

the classifier came back with:3, the real answer is: 3
the classifier came back with:2, the real answer is: 2
the classifier came back with:1, the real answer is: 1
the classifier came back with:1, the real answer is: 1
the classifier came back with:1, the real answer is: 1
the classifier came back with:1, the real answer is: 1
the classifier came back with:3, the real answer is: 3
the classifier came back with:3, the real answer is: 3
the classifier came back with:1, the real answer is: 1
the classifier came back with:3, the real answer is: 3
the classifier came back with:1, the real answer is: 1
the classifier came back with:1, the real answer is: 1
the classifier came back with:2, the real answer is: 2
the classifier came back with:1, the real answer is: 1
the classifier came back with:1, the real answer is: 1
the classifier came back with:1, the real answer is: 1
the classifier came back with:1, the real answer is: 1
the classifier came back with:1, the real answer is: 1
the classi

In [7]:
def classifyPerson():
    resultList = ['not at all','in small doses', 'in large doses']
    percentTats = float(input("percentage of time spent playing video games?"))
    ffMiles = float(input("frequent flier miles earned per year?"))
    iceCream = float(input("liters of ice cream consumed per year?"))
    datingDataMat,datingLabels = file2matrix('datingTestSet2.txt')
    normMat,ranges,minVals = autoNorm(datingDataMat)
    inArr = array([ffMiles,percentTats,iceCream])
    classifierResult = classify0((inArr-minVals)/ranges,normMat,datingLabels,3)
    print("You will probably like this person:",resultList[classifierResult-1])
classifyPerson()

percentage of time spent playing video games?23.123
frequent flier miles earned per year?21512
liters of ice cream consumed per year?2.151
You will probably like this person: in large doses
